# School Ofsted Performance

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


## Section 1: Business Understanding

#### Brief Description:
This section provides an overview of the project's goals and the specific questions it aims to answer about Ofsted ratings and school performance.

###### Question 1: Which shool types have the highest Ofsted rating?

###### Question 2: Which schools by gender enrolement have the highest Ofsted ratings?

###### Question 3: Which schools by religious charter have the highest Ofsted ratings?

###### Question 4: What is the average % of absense for schools that require improvement by Ofsted?

#### Predicting Excellence: The Hidden Patterns in Ofsted Ratings
This notebook will explore the hidden patterns in Ofsted ratings, focusing on key factors influencing ratings, characteristics of high-performing schools. Using data analysis and visualization techniques, we'll delve into the data to answer specific questions about school performance and Ofsted ratings. We evaluate the performance of machine learning models in predicting Ofsted ratings, to understand a concept of whether Ofsted success is deterministic.


### Gather Data
Loading absense data, ofsted data and school population data (demographics) and their relevant data dictionaries


In [ ]:
def get_rawdata(filename):

    """
    This function takes a filename as input and returns a pandas dataframe.

    Parameters:
    filename (string): The file name for the data.

    Returns:
    (pandas.DataFrame): The dataframe
    """

    file_path = os.path.join(
        '/Users/abiibrahim/Documents/2022-2023', filename)
    return pd.read_csv(file_path)


absence = get_rawdata('england_abs.csv').drop(
    columns=['LA', 'ESTAB'])


Here I have removed all categorical columns with large numbers of unique values that would lead to nonsensical results in the ML model.
They have been removed to avoid overfitting

In [ ]:
ofsted = get_rawdata('england_school_information.csv').drop(
    columns=['LANAME', 'LA', 'ESTAB', 'LAESTAB', 'SCHSTATUS', 'OPENDATE',
             'SCHNAME', 'STREET', 'LOCALITY', 'ADDRESS3',
             'TOWN', 'POSTCODE', 'OFSTEDLASTINSP',
             'ISPRIMARY', 'ISSECONDARY', 'ISPOST16', 'AGELOW',
             'AGEHIGH',  'CLOSEDATE', 'OFSTEDLASTINSP'])


In [ ]:
demographics = get_rawdata('2022-2023_england_census.csv')


In [ ]:
def get_dictionary(filename):
    
    """
    This function takes a filename as input and returns a pandas dataframe.
    Used specifically for dictionaries 
    
    Parameters:
    filename (string): The file name for the data.

    Returns:
    (pandas.DataFrame): The dataframe
    """

    file_path = os.path.join(
        '/Users/abiibrahim/Documents/2022-2023 Data dictionary',
        filename)
    return pd.read_csv(file_path, index_col=False)


In [ ]:
schema_absence = get_dictionary('abs_meta.csv')
schema_ofsted = get_dictionary('school_information_meta.csv')
schema_demographics = get_dictionary('2022-2023_census_meta.csv')


In [ ]:
schema_absence.rename(columns={
    'Variable':'Field Name',
    'Label':'Description'}, inplace=True)


In [ ]:
schema_demographics.rename(columns={'Field Reference':'Field Name',
                                    'Field Name':'Description'}, inplace=True)


In [ ]:
schema_absence = schema_absence.set_index('Field Name')
schema_ofsted = schema_ofsted.set_index('Field Name')
schema_demographics = schema_demographics.set_index('Field Name')


## Section 2: Data Understanding

## Data Exploration
Let's start by exploring the dataset to understand its structure and main characteristics.

### Basic Information
We will check the dataset's basic information, such as the number of entries, column names, and data types.


First, let's examine the basic structure of the dataset, including the number of entries, column names, and data types. This information will give us a preliminary understanding of what the data looks like.


### Assess Data

In [ ]:
schema_absence


In [ ]:
schema_ofsted


In [ ]:
schema_demographics


### Clean and join data

In [ ]:
# Here I have dropped identifier columns to prevent overfitting
demographics.drop(columns=['LA', 'Estab'])


The shape of each of the columns we hope to aggregate

In [ ]:
print(absence.shape, ofsted.shape, demographics.shape)

Here I have dropped duplicate rows with the same unique identifier (URN)
URN is the primary key in this data for each school that is investigated

In [ ]:
def global_clean(df):

    """
    This function takes a dataframe as input and returns a non duplicate.

    Parameters:
    df (pandas.DataFrame): The dataframe.

    Returns:
    df_clean (pandas.DataFrame): The dataframe with duplicates checked
    and removed.
    """

    print(f'{df.duplicated().sum()} duplicates found by row,' +
          f'however we have {df.URN.duplicated().sum()} duplicates of URNs.')
    df = df.drop_duplicates(subset = ['URN'])
    df_clean = df.set_index('URN')
    return df_clean


Now I can check the number of duplicates of URNs for each dataframe

In [ ]:
global_clean(absence)
global_clean(ofsted)
global_clean(demographics)


I want to create a main dataframe to use for analysis and modelling, hence I am joining them based on their URNs

In [ ]:
df = global_clean(absence).join(global_clean(ofsted), lsuffix='left')

df = df.join(global_clean(demographics), rsuffix='right')


Let's check the joined table looks as expected with URN as an index column, and the shape is correct.

In [ ]:
df.head()


### Assess main datafram

Display summary statistics and datatypes

In [ ]:
df.info()


The datatypes produced are as expected. 
Now let's check summary statistics

In [ ]:
df.describe()


As 'PERCTOT' and 'PPERSABS10' are both percentages the maximum is as expected which is <= 100.
There is an average absense of 6.8% aswell

In [ ]:
num_rows = df.shape[0] # Provide the number of rows in the dataset
num_cols = df.shape[1] # Provide the number of columns in the dataset

print(num_rows, num_cols)

The number of rows and columns in this dataset are as expected

Which columns had no missing values? Here is a set of column names that have no missing values.

In [ ]:
no_nulls = set(df.columns[df.isnull().mean()==0]) 
# Provides a set of columns with 0 missing values.
no_nulls

Only absense data has been completely reported by .gov/uk

Which columns have the most missing values? Here is a set of column name that have more than 75% if their values missing.

In [ ]:
most_missing_cols = set(df.columns[df.isnull().mean() > 0.75]) 
#Provide a set of columns with more than 75% of the values missing
most_missing_cols

Which columns have the most missing values? Here is a set of column name that have all values missing.

In [ ]:
most_missing_cols = set(df.columns[df.isnull().mean()==1])
#Provide a set of columns with more than 75% of the values missing
most_missing_cols

It is the same columns, which are completely missing. All these columns will be dropped in Data Preperation

This is later used in section 5 to give visualisations.

In [ ]:
def categorical_plot(cat_column):

    """
    This function allows us to quickly plot categorical data
    against the Ofsted rating

    Parameters:
    cat_column (string): A categorical column.

    Returns:
    plot (): A horizontal bar plot
    """

    # Grouping by 'OFSTEDRATING' and counting 'SCHOOLTYPE'
    grouped_data = df.groupby(
        'OFSTEDRATING')[cat_column].value_counts(
        ).unstack().fillna(0)

    # Standardize by the total number of each school type
    total_counts = df[cat_column].value_counts()
    normalized_data = grouped_data.div(total_counts, axis=1)

    # Plotting the bar plot
    normalized_data.plot(kind='barh', stacked=True, figsize=(10, 6))
    plt.title(f'Proportion of {cat_column} by Ofsted Rating')
    plt.xlabel('Ofsted Rating')
    plt.ylabel('Proportion of Schools')
    plt.legend(title=f'{cat_column}')
    plt.show()


This also filters the data if the categorical list is too long to the top 10 values

In [ ]:
def categorical_plot_filtered(cat_column, df):
    """
    This function allows us to quickly plot categorical
    data against the Ofsted rating.

    Parameters:
    cat_column (string): A categorical column name in the dataframe `df`.
    df (DataFrame): The dataframe containing the data.

    Returns:
    plot (): A horizontal bar plot
    """

    # Grouping by 'OFSTEDRATING' and counting 'cat_column'
    grouped_data = df.groupby(
        'OFSTEDRATING')[cat_column].value_counts().unstack().fillna(0)

    # Standardize by the total number of each category in 'cat_column'
    total_counts = df[cat_column].value_counts()
    normalized_data = grouped_data.div(total_counts, axis=1)

    # Get top 10 categories in 'cat_column' by total count
    top_categories = total_counts.nlargest(10).index

    # Filter to include only top 10 categories in 'cat_column'
    filtered_grouped_df = normalized_data[top_categories]

    # Plotting the bar plot
    filtered_grouped_df.plot(kind='barh', stacked=True, figsize=(10, 6))

    plt.title(f'Proportion of {cat_column} by Ofsted Rating')
    plt.xlabel('Proportion of Schools')
    plt.ylabel('Ofsted Rating')
    plt.legend(title=f'{cat_column}')
    plt.show()


Display description for columns if you need to execute a quick Vlookup

In [ ]:
def get_description(column_name, schema):

    """
    This function allows us to lookup the description of column variables

    Parameters:
    column_name (string): The column.
    schema (pandas.DataFrame): The data dictionary

    Returns:
    (string): The description from the schema.
    """

    return schema.loc(
        [schema['Field Name'] == column_name,'Description'].values[0])


search_list = [col for col in df.columns if 'ofsted' in col]
for col in search_list:
    print(get_description(col, schema_ofsted))
    

## Section 3: Data Preperation

Drop rows where target value is empty as that would cause errors with modelling.
Proper handling ensures the integrity of the dataset.

Preprocessing is a critical step in any machine learning project. It involves cleaning and transforming the raw data into a suitable format for modeling. This section covers various preprocessing techniques tailored to our dataset.

In [ ]:
# Dropped entirely empty rows
df = df.dropna(how='all')
# Drop entirely empty columns
df = df.dropna(how='all', axis=1)

# Target Column - Ofsted rating
df = df.dropna(subset=['OFSTEDRATING'])
print(df.shape)


QUICK GLANCE Target Column - Ofsted rating

In [ ]:
df.groupby('MINORGROUP')['OFSTEDRATING'].value_counts()
df['OFSTEDRATING'].value_counts()


Ofsted usually works in 4 system tiered rating, so lets add this to the data frame


In [ ]:
# Adding tiered rating to data frame
def ofsted_tier(rating):
    """
    This function returns the 4-tiered rating of
    Ofsted scores.

    Parameters:
    rating (string): Rating column.

    Returns:
    output (string): The rating measured by the 4-tier
    scale
    """

    if rating == 'Outstanding':
        output = 1
    elif rating == 'Good':
        output = 2
    elif rating == 'Requires improvement':
        output = 3
    else:
        output = 4
    return output


df['OFSTEDGRADE'] = df['OFSTEDRATING'].apply(
    lambda x: ofsted_tier(x))


Creating categorical and numeric columns to isolate how we deal with null values

In [ ]:
numer_cols = df.select_dtypes(['int64', 'float64'])

cat_cols = df.select_dtypes(['object'])

print(numer_cols.shape, cat_cols.shape)


In [ ]:
# Make a scatter matrix of numeric columns
pd.plotting.scatter_matrix(numer_cols, figsize=(15, 10), alpha=0.5)
plt.show()


In [ ]:
# No null values found in numeric columns
num_finder = pd.DataFrame(numer_cols.isnull().sum()).reset_index()
num_finder


In [ ]:
cat_finder = pd.DataFrame(cat_cols.isnull().sum()).reset_index()
cat_finder


### Missing value analysis
Analysing which null features to keep in our categorical dataset
This search list shows all categorical columns with null values that could potentially be imputed

However, chose not to impute these values and kept the null column once the data was unpivoted using getdummies

It is appropriate to keep all nulls related to 'RELCHAR' 'ADMPOL' as the number of nulls are too high to remove, none cause a weakness in the model by keeping them


**Note no imputation was needed.

In [ ]:
search_list = cat_finder.loc[cat_finder[0] != 0, 'index'].values
print(search_list)


In [ ]:
def get_description(column_name, schema):

    """
    This function allows us to lookup the description of column variables

    Parameters:
    column_name (string): The column.
    schema (pandas.DataFrame): The data dictionary

    Returns:
    desc (string): The description from the schema.
    """

    return schema.loc[schema['Field Name'] == column_name,
                             'Description'].values[0]


search_list = [col for col in df.columns if 'ofsted' in col]
for col in search_list:
    print(get_description(col, schema_ofsted))

None of these information requires nulls to be removed, they may still be useful in modelling.

Therefore, missing values are also handled appropriately for both descriptive and ML techniques.

Leave drop_first to true to avoid
Multicollinearity which gives nonsensical R2 values.

In [ ]:
# Create dummies for cat_cols
dummy_df = pd.get_dummies(cat_cols, prefix_sep='_',
                          dummy_na=True, drop_first=True)


In [ ]:
df_clean = dummy_df.join(numer_cols, how='outer').reset_index()
df_clean.head()


Let's check that we have removed al objects before we begin modeling

In [ ]:
df_clean.dtypes.value_counts()


In [ ]:
# Check for potential target columns for binary classification
[col for col in df_clean.columns if 'OFSTEDRATING' in col]


## Section 4: Data Modelling

### Initialising Model

In [ ]:
# Model training
df_clean1 = df_clean.drop(columns=['URN',
                                   'OFSTEDRATING_Outstanding',
                                   'OFSTEDRATING_Requires improvement',
                                   'OFSTEDRATING_Special Measures',
                                   'OFSTEDRATING_nan',
                                   'OFSTEDGRADE']).copy()

target_column = 'OFSTEDRATING_Serious Weaknesses'

X = df_clean1.drop(columns=[target_column]).copy()
y = df_clean1[target_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.3)

lr = LinearRegression(
    fit_intercept=True,
    copy_X=True,
    n_jobs=None,
    positive=False)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)


### Model Performance

In [ ]:
# Model evaluation
r2 = (r2_score(y_test, y_pred))
print("R^2 score:", r2)

The performance is nonsensical using linear regression, so now we try randomforest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Fit Random Forest model
rf_model = RandomForestRegressor(n_estimators=20)

rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = rf_model.predict(X_test)
print("R^2 score:", r2_score(y_test, y_pred_rf))


More reasonable scores, however still a weak performance

## Section 5: Evaluate the Results

#### 1) Which shool types have the highest Ofsted rating?

In [ ]:
categorical_plot('ADMPOL')


In [ ]:
categorical_plot_filtered('SCHOOLTYPE',df)


From this evaluation, Selective schools are awarded substantially more Outstanding scores for their Ofsted reports. 
Likewise non-selective schools are more likely to be scored as a serious weakness.

Academy special converter schools and Community special schools perform well in Ofsted inspections. 

In [ ]:
# Taking a closer look into those schools under grade 4 rating
categorical_plot_filtered('SCHOOLTYPE',df.loc[df['OFSTEDGRADE'] == 4])


Free schools and Academy converters on average perform worse in Ofsted inspections.

#### 2) Which schools by gender enrolment have the highest Ofsted ratings?


In [ ]:
categorical_plot('GENDER')

From here we can understand all girls schools outperform all-boys schools.
Additionally single gender schools perform better getting either an 'Outstanding' or 'Good' report from Ofsted compared to mixed-gender schools. Potentially absences could also explain the differences in scores. Suprisingly, mixed schools have lower absences.

In [ ]:
df.groupby('GENDER')['PERCTOT'].mean()


Here we can evaluate the percentage of school-tyoes with persistent absenteeism > 10% for each student.

In [ ]:
df.groupby('GENDER')['PPERSABS10'].mean()


#### 3) Are there differences in Ofsted ratings based on religious charter?

In [ ]:
categorical_plot_filtered('RELCHAR', df)


Multiple conclusions can be made from here.
Muslim community schools perform substantially better in Ofsted ratings than non-religious chartered schools.
The Church of England makes a large proportion of schools that require improvement

#### 4) What is the average % of absense for schools that require improvement by Ofsted?

In [ ]:
df_clean1.PERCTOT.mean()


The average percentage of overall absense in schools within the UK for 2022-23 was 6.78%.

The average absence for schools requiring improvement on average are once their >8%.

In [ ]:
df.groupby('OFSTEDRATING')['PERCTOT'].mean()


In [ ]:
# Plotting using seaborn
plt.figure(figsize=(10, 6))
sns.stripplot(x='ADMPOL', y='PERCTOT',
              hue='SCHOOLTYPE', data=df, jitter=True, dodge=True, linewidth=1)
plt.title('Percentage of Absences by Ofsted Rating')
plt.xlabel('Ofsted Rating')
plt.ylabel('Percentage of Absences')
plt.legend(title='School Type')
plt.show()


Taking into account absense can help in concluding that this factor has a large influence on the feedback that Ofsted gives.
From the graph above non-selective schools have much higher absenses than selective schools which may indicate the differences in final Ofsted reports recieved back. 

## Section 6: Conclusions

My motivation to carry out this research was to understand if Ofsted inspections in the UK are influenced by intrinsic school characteristics as opposed to the actual observed quality of teaching.

Through a comprehensive analysis of Ofsted inspection reports, school demographic data, and educational outcomes, this study was able to determine whether factors such as school type, socio-economic status, and historical performance play a deterministic role in Ofsted ratings. The findings aim to contribute to the ongoing debate on the fairness and effectiveness of educational evaluations and provide recommendations for policy improvements.

The machine learning model itself was weak in confirming these characteristics determined the Ofsted rating. 

It also suggests the invaluablity of Ofsted reports. It does not mirror standard indicators nor can it replace league tables. Instead, they offer a valuable assessment of the quality of education provided. Schools have significant agency in influencing their ratings through high-quality teaching and effective management. These findings underscore the importance of continuous improvement in educational practices and reaffirm the credibility of Ofsted as a measure of school performance.
